In [1]:
# Uninstall potentially conflicting packages
!pip uninstall -y transformers accelerate unsloth torch torchvision torchaudio

# Install base packages
!pip install unsloth

# Install dependencies
!pip install -q transformers accelerate peft
!pip install -q datasets evaluate bitsandbytes trl
!pip install -q torch torchvision torchaudio

# Install Colab-optimized unsloth
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Install other tools
!pip install pandas scikit-learn
!pip install -q ipywidgets

Found existing installation: transformers 4.46.3
Uninstalling transformers-4.46.3:
  Successfully uninstalled transformers-4.46.3
Found existing installation: accelerate 1.1.1
Uninstalling accelerate-1.1.1:
  Successfully uninstalled accelerate-1.1.1
Found existing installation: torch 2.5.1+cu121
Uninstalling torch-2.5.1+cu121:
  Successfully uninstalled torch-2.5.1+cu121
Found existing installation: torchvision 0.20.1+cu121
Uninstalling torchvision-0.20.1+cu121:
  Successfully uninstalled torchvision-0.20.1+cu121
Found existing installation: torchaudio 2.5.1+cu121
Uninstalling torchaudio-2.5.1+cu121:
  Successfully uninstalled torchaudio-2.5.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.2/174.2 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 104.9 MB/s eta 0:00:00
Found existing installation: unsloth 2024.12.4
Uninstalling unsloth-2024.12.4:
  Successfully uninstalled unsloth-2024.12.4
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-h7t18g5n/unsloth_b8ddca38ab0e4adaa723e8d0d9cd3709
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-h7t18g5n/unsloth_b8ddca38ab0e4adaa723e8d0d9cd3709
  Resolved https://github.com/unslothai/unsloth.git to commit 85f1fa096afde5efe2fb8521d8ceec8d13a00715
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2024.12.4-py3-none-any.whl size=173746 sha256=51148df01bfbd75c543188db2c376

Preparations
------

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Environment setup
import os
import warnings
import random
import numpy as np
import torch
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import pandas as pd
import gc
from unsloth import FastLanguageModel
from trl import SFTTrainer
from transformers import TrainingArguments, TrainerCallback
import transformers
import accelerate
import json

# Print versions
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")

# Configure environment
os.environ["CUDA_VISIBLE_DEVICES"]="0"
warnings.filterwarnings('ignore')
torch.set_float32_matmul_precision('high')

# Set random seeds
def set_seeds(seed=3407):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

# Memory management utilities
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()

def print_gpu_utilization():
    print("\nGPU Memory Usage:")
    !nvidia-smi | grep -E "Memory|Volatile"

def print_detailed_gpu_info():
    print("\nDetailed GPU Memory Info:")
    print(f"Allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
    print(f"Cached: {torch.cuda.memory_reserved()/1024**2:.2f} MB")
    print(f"Max Allocated: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")

Mounted at /content/drive
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
PyTorch version: 2.5.1+cu124
Transformers version: 4.46.3
Accelerate version: 1.2.1


set up wandb

In [3]:
!pip install wandb

import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

define the sweep

In [4]:
sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'loss',
    'goal': 'minimize'
    }

sweep_config['metric'] = metric

parameters_dict = {
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
        },
    'warmup_ratio': {
        'values': [0.05, 0.1, 0.2]
        },
    'weight_decay': {
        'values': [0.01, 0.03, 0.05]
        },
    'per_device_train_batch_size': {
        'values': [2, 4]
        },
    'gradient_accumulation_steps': {
        'values': [2, 4, 8]
        },
    'epochs': {
        'value': 1
        }
}

sweep_config['parameters'] = parameters_dict
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'epochs': {'value': 1},
                'gradient_accumulation_steps': {'values': [2, 4, 8]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.001,
                                  'min': 1e-05},
                'per_device_train_batch_size': {'values': [2, 4]},
                'warmup_ratio': {'values': [0.05, 0.1, 0.2]},
                'weight_decay': {'values': [0.01, 0.03, 0.05]}}}


In [5]:
import pprint

pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'loss'},
 'parameters': {'epochs': {'value': 1},
                'gradient_accumulation_steps': {'values': [2, 4, 8]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.001,
                                  'min': 1e-05},
                'per_device_train_batch_size': {'values': [2, 4]},
                'warmup_ratio': {'values': [0.05, 0.1, 0.2]},
                'weight_decay': {'values': [0.01, 0.03, 0.05]}}}


Define class
---

In [6]:
class MemoryCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        if state.global_step % 50 == 0:  # 每50步清理一次
            clear_memory()
            print_detailed_gpu_info()

class AIGenerationDetector:
    def __init__(self, max_seq_length=2048, save_dir='/content/drive/MyDrive/ai_detection_model', data_dir='/content/drive/MyDrive/ai_dataset'):
        """
        Initializes the AIGenerationDetector class.
        Args:
            max_seq_length (int): Maximum sequence length for the model.
            save_dir (str): Directory to save the model and checkpoints.
            data_dir (str): Directory to save the dataset.
        """
        self.max_seq_length = max_seq_length
        self.save_dir = save_dir
        self.data_dir = data_dir
        self.model = None
        self.tokenizer = None
        self.train_dataset = None
        self.eval_dataset = None
        self.test_dataset = None
        os.makedirs(self.save_dir, exist_ok=True)
        os.makedirs(self.data_dir, exist_ok=True)


    def download_data(self):
        """
        Downloads the dataset from GitHub and saves to Google Drive.
        """

        github_base_url = "https://raw.githubusercontent.com/botianzhe/CHEAT/main/data/"
        file_names = [
            "ieee-chatgpt-fusion.jsonl",
            "ieee-chatgpt-generation.jsonl",
            "ieee-init.jsonl",
            "ieee-chatgpt-polish.jsonl",
            "ieee-chatgpt-fusion.xlsx",
            "ieee-chatgpt-generation.xlsx",
            "ieee-init.xlsx",
            "ieee-chatgpt-polish.xlsx"
        ]
        print("Downloading data from Github...")
        for file_name in file_names:
              file_url = github_base_url + file_name
              output_path = os.path.join(self.data_dir, file_name) # saving location

              if not os.path.exists(output_path): # download only when the data does not exist.
                print(f"Downloading {file_name}...")
                !wget "{file_url}" -O "{output_path}" # using wget for direct download
              else:
                 print(f"File {file_name} already exists. Skipping...")
        print("All data downloaded!")

    def setup_model(self):
        """
        Loads the pre-trained model and tokenizer and configures the PEFT model.
        """
        clear_memory()
        print("Loading model...")

        try:
            model, tokenizer = FastLanguageModel.from_pretrained(
                model_name="unsloth/Meta-Llama-3.1-8B",
                max_seq_length=self.max_seq_length,
                load_in_4bit=True,
            )

            model = FastLanguageModel.get_peft_model(
                model,
                r=16,
                target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
                lora_alpha=16,
                lora_dropout=0.1,
                bias="none",
                use_gradient_checkpointing=True,
                random_state=3407,
                use_rslora=True,
            )

            self.model = model
            self.tokenizer = tokenizer
            print("Model loaded successfully!")

        except Exception as e:
            print(f"Error loading model: {str(e)}")
            raise

    def prepare_datasets(self, max_samples=1000):
        """
        Prepares the training and evaluation datasets.
        Args:
            max_samples (int): Maximum number of training samples to use.
        """
        clear_memory()
        print("Preparing datasets...")
        try:
            all_data = []  # Initialize a list to hold all training examples.

            # Data holders for each jsonl file
            init_data = {}
            generation_data = {}
            polish_data = {}
            fusion_data = []

            # Load data from each file
            for filename in os.listdir(self.data_dir):
                if filename.endswith("init.jsonl"):
                    filepath = os.path.join(self.data_dir, filename)
                    with open(filepath, 'r', encoding='utf-8') as f:
                        for line in f:
                            try:
                                data_item = json.loads(line)
                                init_data[data_item['id']] = data_item['abstract']  # use id as key
                            except json.JSONDecodeError:
                                print(f"Warning: Skipping invalid JSON line in {filename}")

                elif filename.endswith("generation.jsonl"):
                    filepath = os.path.join(self.data_dir, filename)
                    with open(filepath, 'r', encoding='utf-8') as f:
                        for line in f:
                            try:
                                data_item = json.loads(line)
                                generation_data[data_item['id']] = data_item['abstract']
                            except json.JSONDecodeError:
                                print(f"Warning: Skipping invalid JSON line in {filename}")


                elif filename.endswith("polish.jsonl"):
                    filepath = os.path.join(self.data_dir, filename)
                    with open(filepath, 'r', encoding='utf-8') as f:
                        for line in f:
                            try:
                                data_item = json.loads(line)
                                polish_data[data_item['id']] = data_item['abstract']
                            except json.JSONDecodeError:
                                print(f"Warning: Skipping invalid JSON line in {filename}")

                elif filename.endswith("fusion.jsonl"):
                    filepath = os.path.join(self.data_dir, filename)
                    with open(filepath, 'r', encoding='utf-8') as f:
                        for line in f:
                            try:
                                data_item = json.loads(line)
                                fusion_data.append(data_item)
                            except json.JSONDecodeError:
                                print(f"Warning: Skipping invalid JSON line in {filename}")


            # Create training data by pairing the init, generation, and polish abstracts
            common_ids = set(init_data.keys()) & set(generation_data.keys()) & set(polish_data.keys())
            for id in common_ids:
                all_data.append({'text': init_data[id], 'is_ai_generated': "False"}) # human written data
                all_data.append({'text': generation_data[id], 'is_ai_generated': "True"}) # ai generated data
                all_data.append({'text': polish_data[id], 'is_ai_generated': "True"}) # ai generated data

            # Create a Dataset from list of dicts.
            dataset = Dataset.from_list(all_data)

            # Take required number of samples, if specified
            if max_samples > 0:
                dataset = dataset.shuffle(seed=3407).select(range(max_samples))

            # Split train and val set
            train_idx, val_idx = train_test_split(
                range(len(dataset)),
                test_size=0.1,
                random_state=3407
            )

            train_examples = [self.process_training_example(dataset[i]) for i in train_idx]
            eval_examples = [self.process_training_example(dataset[i]) for i in val_idx]

            self.train_dataset = Dataset.from_list(train_examples)
            self.eval_dataset = Dataset.from_list(eval_examples)

            del train_examples, eval_examples, dataset
            clear_memory()
            print(f"Datasets prepared! Train size: {len(self.train_dataset)}, Eval size: {len(self.eval_dataset)}")


        except Exception as e:
            print(f"Error preparing datasets: {str(e)}")
            raise

    def create_test_dataset(self):
        """
        Prepares the test dataset from fusion data.
        """
        clear_memory()
        print("Preparing test dataset...")
        try:
            fusion_data = []

            for filename in os.listdir(self.data_dir):
                if filename.endswith("fusion.jsonl"):
                    filepath = os.path.join(self.data_dir, filename)
                    with open(filepath, 'r', encoding='utf-8') as f:
                        for line in f:
                            try:
                                data_item = json.loads(line)
                                if 'abstract' in data_item:
                                    fusion_data.append({'text': data_item['abstract']})
                                else:
                                    print(f"Warning: Skipping entry with no 'abstract' in {filename}")
                            except json.JSONDecodeError:
                                print(f"Warning: Skipping invalid JSON line in {filename}")

            self.test_dataset = Dataset.from_list(fusion_data)
            print(f"Test dataset prepared! Size: {len(self.test_dataset)}")

        except Exception as e:
            print(f"Error preparing test dataset: {str(e)}")
            raise


    def process_training_example(self, example):
         """
         Processes a single training example to create the prompt.
         Args:
            example (dict): A dictionary containing the 'text' and 'is_ai_generated' fields.
         Returns:
            dict: A dictionary containing the processed text.
         """
         text = example['text']
         is_ai_generated = example['is_ai_generated']  # Get the field from the dataset

         prompt = (
            "You are an expert in distinguishing between text written by humans and text generated by AI.\n\n"
            f"Given Text: {text}\n\n"
            "Based on careful analysis, is the text generated by an AI? Respond with EXACTLY 'True' or 'False'.\n"
            f"Answer: {str(is_ai_generated)}"
         ) + self.tokenizer.eos_token

         return {"text": prompt}


    def process_test_example(self, example):
        """
        Processes a single test example to create the prompt.
        Args:
            example (dict): A dictionary containing the 'text' field.
        Returns:
            str: The generated prompt for testing.
        """
        text = example['text']

        prompt = (
           "You are an expert in distinguishing between text written by humans and text generated by AI.\n\n"
           f"Given Text: {text}\n\n"
           "Based on careful analysis, is the text generated by an AI? Respond with EXACTLY 'True' or 'False'.\n"
        )
        return prompt

    def setup_training_args(self, config=None):
        """
        Sets up training arguments, either default or for hyperparameter sweeping.
        Args:
           config (dict, optional): Configuration for hyperparameter sweep. Defaults to None.
        Returns:
           TrainingArguments: Training arguments based on the given configuration.
        """
        if config is None:
            # Default training arguments
            return TrainingArguments(
                output_dir=os.path.join(self.save_dir, "checkpoints"),
                per_device_train_batch_size=2,
                gradient_accumulation_steps=8,
                warmup_ratio=0.1,
                num_train_epochs=3,
                learning_rate=0.0006026,
                fp16=True,
                logging_steps=10,
                optim="adamw_torch",
                weight_decay=0.05,
                lr_scheduler_type="cosine",
                seed=3407,
                evaluation_strategy="steps",
                eval_steps=50,
                save_strategy="steps",
                save_steps=50,
                load_best_model_at_end=True,
                metric_for_best_model="eval_loss",
                gradient_checkpointing=True,
                max_grad_norm=0.3,
                report_to="none",
                remove_unused_columns=True,
                dataloader_pin_memory=False
            )
        else:
            # Training arguments for hyperparameter sweep
            return TrainingArguments(
                output_dir=os.path.join(self.save_dir, "checkpoints"),
                per_device_train_batch_size=config.per_device_train_batch_size,
                gradient_accumulation_steps=config.gradient_accumulation_steps,
                warmup_ratio=config.warmup_ratio,
                num_train_epochs=config.epochs,
                learning_rate=config.learning_rate,
                fp16=True,
                logging_steps=10,
                optim="adamw_torch",
                weight_decay=config.weight_decay,
                lr_scheduler_type="cosine",
                seed=3407,
                evaluation_strategy="steps",
                eval_steps=50,
                save_strategy="steps",
                save_steps=50,
                load_best_model_at_end=True,
                metric_for_best_model="eval_loss",
                gradient_checkpointing=True,
                max_grad_norm=0.3,
                report_to="wandb",
                remove_unused_columns=True,
                dataloader_pin_memory=False,
            )

    def train(self):
        """
        Trains the model using the SFTTrainer.
        """
        clear_memory()
        print("Starting training...")

        try:
            trainer = SFTTrainer(
                model=self.model,
                tokenizer=self.tokenizer,
                train_dataset=self.train_dataset,
                eval_dataset=self.eval_dataset,
                dataset_text_field="text",
                max_seq_length=self.max_seq_length,
                dataset_num_proc=2,
                packing=False,
                args=self.setup_training_args(),
                callbacks=[MemoryCallback()]
            )

            trainer.train()

            final_save_path = os.path.join(self.save_dir, "final_model")
            self.model.save_pretrained(final_save_path)
            self.tokenizer.save_pretrained(final_save_path)
            print(f"Training completed! Model saved to {final_save_path}")

        except Exception as e:
            print(f"Error during training: {str(e)}")
            raise


    def sweep(self):
       """
        Conducts a hyperparameter sweep using Weights & Biases.
        """
       print("Starting sweeping...")

       with wandb.init():
            config = wandb.config
            training_args = self.setup_training_args(config)

            try:
                trainer = SFTTrainer(
                    model=self.model,
                    tokenizer=self.tokenizer,
                    train_dataset=self.train_dataset,
                    eval_dataset=self.eval_dataset,
                    dataset_text_field="text",
                    max_seq_length=self.max_seq_length,
                    dataset_num_proc=2,
                    packing=False,
                    args=training_args
                    )
                trainer.train()

                final_save_path = os.path.join(self.save_dir, "final_model")
                self.model.save_pretrained(final_save_path)
                self.tokenizer.save_pretrained(final_save_path)
                print(f"Training completed! Model saved to {final_save_path}")

            except Exception as e:
                  print(f"Error during training: {str(e)}")
                  raise


    def generate_predictions(self, batch_size=16):
         """
        Generates predictions on the test dataset.
        Args:
           batch_size (int): Batch size for generating predictions.
        Returns:
           list: List of prediction values.
        """
         clear_memory()
         print("Generating predictions...")

         try:
            FastLanguageModel.for_inference(self.model)
            predictions = []
            # Convert test data to a list to support batch processing
            test_examples = list(self.test_dataset)
            total_batches = (len(test_examples) + batch_size - 1) // batch_size
            all_predictions = []

            # Process in batches
            for i in range(0, len(test_examples), batch_size):
                if i % (batch_size * 10) == 0:
                    print(f"Processing batch {i//batch_size}/{total_batches}")

                # Get current batch samples
                batch = test_examples[i:i + batch_size]
                prompts = [self.process_test_example(example) for example in batch]

                # Batch encoding
                inputs = self.tokenizer(
                    prompts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=self.max_seq_length
                ).to("cuda")

                # Batch generation
                with torch.inference_mode():
                    outputs = self.model.generate(
                        **inputs,
                        max_new_tokens=8,  # Reduce the number of generated tokens as we only need True/False
                        temperature=0.1,
                        top_p=0.9,
                        do_sample=False,    # Disable sampling for faster generation
                        use_cache=True,
                        pad_token_id=self.tokenizer.pad_token_id,
                    )

                input_length = inputs['input_ids'].shape[1]
                responses = self.tokenizer.batch_decode(
                    [output[input_length:] for output in outputs],
                    skip_special_tokens=True
                )

                # Batch processing prediction results
                batch_predictions = ["true" in response.lower() for response in responses]
                all_predictions.extend(batch_predictions)

                # Periodically clear memory
                if i % (batch_size * 50) == 0:
                    clear_memory()

            print(f"Total predictions: {len(all_predictions)}")
            assert len(all_predictions) == len(test_examples)

            return all_predictions

         except Exception as e:
            print(f"Error generating predictions: {str(e)}")
            raise


    def create_submission(self):
        """
        Creates the submission file in CSV format using the generated predictions.
        """
        print("Creating submission file...")
        try:
            predictions = self.generate_predictions(batch_size=16)
            print(f"Generated predictions: {len(predictions)}")

            assert len(predictions) == len(self.test_dataset), \
                f"Prediction count mismatch! Expected {len(self.test_dataset)}, got {len(predictions)}"

            submission_df = pd.DataFrame({
                'ID': range(len(predictions)),
                'is_ai_generated': predictions
            })

            print(f"Submission DataFrame shape: {submission_df.shape}")

            submission_path = os.path.join(self.save_dir, 'submission.csv')
            submission_df.to_csv(submission_path, index=False)
            print(f"Submission saved to {submission_path}")

            saved_df = pd.read_csv(submission_path)
            print(f"Saved file shape: {saved_df.shape}")

        except Exception as e:
            print(f"Error creating submission: {str(e)}")
            raise

# Hyper Parameters sweeping

## 1. Initial the sweep

In [7]:
sweep_id = wandb.sweep(sweep_config, project="ai_detection")

Create sweep with ID: xr6gzwrx
Sweep URL: https://wandb.ai/bw2676-new-york-university/ai_detection/sweeps/xr6gzwrx


## 2. Run sweep agent

In [8]:
def run_sweep(data_dir):
    """
    Runs the hyperparameter sweep.
    Args:
      data_dir(str): Path to the data directory.
    """
    trainer = AIGenerationDetector(data_dir = data_dir) # set data directory path
    trainer.download_data()
    trainer.setup_model()
    trainer.prepare_datasets(max_samples=1000)
    trainer.sweep()

In [ ]:
wandb.agent(sweep_id, lambda:run_sweep('/content/drive/MyDrive/ai_dataset'), count = 50)

wandb: Agent Starting Run: pge4yv0w with config:
wandb: 	epochs: 1
wandb: 	gradient_accumulation_steps: 2
wandb: 	learning_rate: 0.0004176073291840377
wandb: 	per_device_train_batch_size: 4
wandb: 	warmup_ratio: 0.1
wandb: 	weight_decay: 0.01


File ieee-chatgpt-fusion.jsonl already exists. Skipping...
File ieee-chatgpt-generation.jsonl already exists. Skipping...
File ieee-init.jsonl already exists. Skipping...
File ieee-chatgpt-polish.jsonl already exists. Skipping...
File ieee-chatgpt-fusion.xlsx already exists. Skipping...
File ieee-chatgpt-generation.xlsx already exists. Skipping...
File ieee-init.xlsx already exists. Skipping...
File ieee-chatgpt-polish.xlsx already exists. Skipping...
All data downloaded!
Loading model...
==((====))==  Unsloth 2024.12.4: Fast Llama patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

...

Map (num_proc=2):   0%|          | 0/900 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 900 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 2
\        /    Total batch size = 4 | Total steps = 225
 "-____-"     Number of trainable parameters = 13,631,488
wandb: WARNING Config item 'per_device_train_batch_size' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'gradient_accumulation_steps' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'learning_rate' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'weight_decay' was locked by 'sweep' (ignored update).
wandb: WARNING Config item 'warmup_ratio' was locked by 'sweep' (ignored update).


Step,Training Loss,Validation Loss
50,1.752200,1.686088
100,1.625200,1.665632


Step,Training Loss,Validation Loss
50,1.752200,1.686088
100,1.625200,1.665632
150,1.674000,1.646945


# Get final result

Before running the main function, hyper parameter in trainer.train() should be changed.

In [ ]:
def main(data_dir):
    """
    Main training and evaluation pipeline.
    Args:
       data_dir (str): Path to data directory.
    """
    try:
        set_seeds()
        print("Starting training pipeline...")

        # Initialize and run trainer
        trainer = AIGenerationDetector(data_dir=data_dir)  # set data directory path
        trainer.download_data()
        trainer.setup_model()
        trainer.prepare_datasets(max_samples=5000)
        trainer.create_test_dataset()
        trainer.train()
        trainer.create_submission()

        print("Training pipeline completed successfully!")

    except Exception as e:
        print(f"Fatal error in main: {str(e)}")
        raise
    finally:
        clear_memory()
        print_gpu_utilization()
        print_detailed_gpu_info()

# Main execution block
if __name__ == "__main__":
    data_dir = "/content/drive/MyDrive/ai_dataset"  # Set the Google Drive path to save data

    # Run the final training using main()
    main(data_dir)